In [1]:
# libraries for EDA
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#To display the plots
%matplotlib inline

# scikit learn library for machine learning algorithms, data preprocessing, and evaluation
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, OrdinalEncoder, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_validate, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import classification_report, confusion_matrix, log_loss, recall_score, accuracy_score, precision_score, f1_score, roc_auc_score
from sklearn.metrics import make_scorer, average_precision_score, PrecisionRecallDisplay, precision_recall_curve, roc_curve
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PowerTransformer
from sklearn.pipeline import Pipeline

import scikitplot as skplt

# yellowbrick library for visualizing the model performance
from yellowbrick.classifier import ConfusionMatrix, PrecisionRecallCurve
from yellowbrick.classifier import PrecisionRecallCurve
from sklearn.pipeline import Pipeline

# to get rid of the warnings
import warnings
warnings.filterwarnings("ignore")
warnings.warn("this will not show")

plt.rcParams["figure.figsize"] = (10,6)
sns.set_style("whitegrid", {'axes.grid' : True})

# To display maximum columns
pd.set_option('display.max_columns', 50)

# To display maximum rows
pd.set_option('display.max_rows', 50)

# To set float format
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [2]:
def eval_metric(model, X_train, y_train, X_test, y_test):   
    y_pred = model.predict(X_test)
    y_train_pred = model.predict(X_train)
    
    print("Test_Set")
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))
    print()
    print("Train_Set")
    print(confusion_matrix(y_train, y_train_pred))
    print(classification_report(y_train, y_train_pred))

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test_x.csv')

df = train.copy()

In [4]:
df['A'] = df['Yıllık Ortalama Sipariş Verilen Ürün Adedi'] / df['Yıllık Ortalama Sepete Atılan Ürün Adedi']
df['B'] = df['Yıllık Ortalama Satın Alım Miktarı'] / df['Yıllık Ortalama Gelir'] 
df['C'] = df['Yıllık Ortalama Satın Alım Miktarı'] / df['Yıllık Ortalama Sipariş Verilen Ürün Adedi']
df["D"]= df['Cinsiyet'] + " " + df['Yaş Grubu'] + " " + df["Medeni Durum"]

In [5]:
test['A'] = test['Yıllık Ortalama Sipariş Verilen Ürün Adedi'] / test['Yıllık Ortalama Sepete Atılan Ürün Adedi'] 
test['B'] = test['Yıllık Ortalama Satın Alım Miktarı'] / test['Yıllık Ortalama Gelir']
test['C'] = test['Yıllık Ortalama Satın Alım Miktarı'] / test['Yıllık Ortalama Sipariş Verilen Ürün Adedi']
test["D"]= test['Cinsiyet'] + " " + test['Yaş Grubu'] + " " + test["Medeni Durum"]

In [6]:
df.head()

,index,Cinsiyet,Yaş Grubu,Medeni Durum,Eğitim Düzeyi,İstihdam Durumu,Yıllık Ortalama Gelir,Yaşadığı Şehir,En Çok İlgilendiği Ürün Grubu,Yıllık Ortalama Satın Alım Miktarı,Yıllık Ortalama Sipariş Verilen Ürün Adedi,Eğitime Devam Etme Durumu,Öbek İsmi,Yıllık Ortalama Sepete Atılan Ürün Adedi,A,B,C
0,0,Kadın,31-40,Bekar,Yüksek Lisans Mezunu,Düzenli ve Ücretli Bir İşi Var,748266.44,Büyük Şehir,Elektronik ve Teknolojik Ürünler,32010.90,37.31,Etmiyor,obek_4,102.94,0.36,0.04,0.04
1,1,Erkek,>60,Evli,Lise Mezunu,Kendi İşinin Sahibi,246298.61,Küçük Şehir,Spor Malzemeleri,4145.73,11.55,Etmiyor,obek_5,82.18,0.14,0.02,0.02
2,2,Erkek,18-30,Bekar,Lise Mezunu,Düzenli ve Ücretli Bir İşi Var,268582.73,Küçük Şehir,Giyim,9924.38,55.35,Ediyor,obek_8,141.66,0.39,0.04,0.04
3,3,Erkek,51-60,Evli,Lise Mezunu,Kendi İşinin Sahibi,327721.84,Kırsal,Giyim,6417.78,15.42,Etmiyor,obek_3,17.10,0.90,0.02,0.02
4,4,Erkek,31-40,Evli,Ortaokul Mezunu,İşsiz veya Düzenli Bir İşi Yok,397431.63,Büyük Şehir,Giyim,7886.66,14.69,Etmiyor,obek_3,20.47,0.72,0.02,0.02


In [7]:
from sklearn.model_selection import train_test_split

X = df.drop(["Öbek İsmi", 'index'], axis=1)
y = df["Öbek İsmi"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [8]:
cat = X_train.select_dtypes("object").columns
cat

Index(['Cinsiyet', 'Yaş Grubu', 'Medeni Durum', 'Eğitim Düzeyi',
       'İstihdam Durumu', 'Yaşadığı Şehir', 'En Çok İlgilendiği Ürün Grubu',
       'Eğitime Devam Etme Durumu'],
      dtype='object')

In [9]:
ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', 
                         unknown_value=-1)

column_trans = make_column_transformer((ord_enc, cat), 
                                        remainder='passthrough')

In [10]:
import lightgbm as lgb

operations_lightgbm = [("OrdinalEncoder", column_trans), 
                       ("lightgbm_model", lgb.LGBMClassifier(random_state=42))]

lightgbm_model = Pipeline(steps=operations_lightgbm)

lightgbm_model.fit(X_train, y_train)

Pipeline(steps=[('OrdinalEncoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  Index(['Cinsiyet', 'Yaş Grubu', 'Medeni Durum', 'Eğitim Düzeyi',
       'İstihdam Durumu', 'Yaşadığı Şehir', 'En Çok İlgilendiği Ürün Grubu',
       'Eğitime Devam Etme Durumu'],
      dtype='object'))])),
                ('lightgbm_model', LGBMClassifier(random_state=42))])

In [11]:
eval_metric(lightgbm_model, X_train, y_train, X_test, y_test)

Test_Set
[[131   1   3   0   0   0   1   2]
 [  0 104   2   0   0   0   1   2]
 [  0   0 135   0   1   3   0   0]
 [  1   0   2 137   0   1   2   1]
 [  1   1   1   1 134   2   0   1]
 [  1   1   0   0   0 133   2   1]
 [  1   2   1   0   0   1 134   0]
 [  0   0   2   2   0   0   2 138]]
              precision    recall  f1-score   support

      obek_1       0.97      0.95      0.96       138
      obek_2       0.95      0.95      0.95       109
      obek_3       0.92      0.97      0.95       139
      obek_4       0.98      0.95      0.96       144
      obek_5       0.99      0.95      0.97       141
      obek_6       0.95      0.96      0.96       138
      obek_7       0.94      0.96      0.95       139
      obek_8       0.95      0.96      0.96       144

    accuracy                           0.96      1092
   macro avg       0.96      0.96      0.96      1092
weighted avg       0.96      0.96      0.96      1092


Train_Set
[[554   0   0   0   0   0   0   0]
 [  0 438   0

In [12]:
from sklearn.metrics import accuracy_score

y_pred = lightgbm_model.predict(X_test)
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

LightGBM Model accuracy score: 0.9579


In [13]:
import lightgbm as lgb

operations_lightgbm = [("OrdinalEncoder", column_trans), 
                       ("lightgbm_model", lgb.LGBMClassifier(boosting_type='dart',
                                                             colsample_bytree=0.6,
                                                             is_unbalance=True,
                                                             learning_rate=0.01,
                                                             min_child_samples=30,
                                                             n_estimators=1000,
                                                             num_leaves=70,
                                                             reg_alpha=0,
                                                             scale_pos_weight=4,
                                                             subsample_for_bin=3000,
                                                             random_state=42))]

lightgbm_model = Pipeline(steps=operations_lightgbm)

lightgbm_model.fit(X_train, y_train)

Pipeline(steps=[('OrdinalEncoder',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ordinalencoder',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  Index(['Cinsiyet', 'Yaş Grubu', 'Medeni Durum', 'Eğitim Düzeyi',
       'İstihdam Durumu', 'Yaşadığı Şehir', 'En Çok İlgilendiği Ürün Grubu',
       'Eğitime Devam Etme Durumu'],
      dtype='object'))])),
                ('lightgbm_model',
                 LGBMClassifier(boosting_type='dart', colsample_bytree=0.6,
                                is_unbalance=True, learning_rate=0.01,
                                min_child_samples=30, n_estimators=1000,
                                num_leaves=70, random_state=42, reg_alpha=0,
                                scale_pos_weight=4, subsample_for_bin=3000))])

In [14]:
eval_metric(lightgbm_model, X_train, y_train, X_test, y_test)

Test_Set
[[132   1   2   0   0   0   1   2]
 [  0 104   2   0   0   0   1   2]
 [  0   0 135   0   1   3   0   0]
 [  1   0   2 137   0   1   2   1]
 [  1   1   1   1 134   2   0   1]
 [  1   1   0   0   0 133   2   1]
 [  1   1   1   0   0   1 135   0]
 [  0   0   2   2   0   0   2 138]]
              precision    recall  f1-score   support

      obek_1       0.97      0.96      0.96       138
      obek_2       0.96      0.95      0.96       109
      obek_3       0.93      0.97      0.95       139
      obek_4       0.98      0.95      0.96       144
      obek_5       0.99      0.95      0.97       141
      obek_6       0.95      0.96      0.96       138
      obek_7       0.94      0.97      0.96       139
      obek_8       0.95      0.96      0.96       144

    accuracy                           0.96      1092
   macro avg       0.96      0.96      0.96      1092
weighted avg       0.96      0.96      0.96      1092


Train_Set
[[531   1   5   7   2   3   4   1]
 [  3 423   2

In [15]:
from sklearn.metrics import accuracy_score

y_pred = lightgbm_model.predict(X_test)
accuracy=accuracy_score(y_pred, y_test)
print('LightGBM Model accuracy score: {0:0.4f}'.format(accuracy_score(y_test, y_pred)))

LightGBM Model accuracy score: 0.9597


In [16]:
predict = lightgbm_model.predict(test)

In [17]:
data = {
    'id': range(2340),
    'Öbek İsmi': predict
}
sub_df = pd.DataFrame(data)
sub_df.set_index('id', inplace=True)
sub_df

,Öbek İsmi
id,
0,obek_3
1,obek_3
2,obek_2
3,obek_6
4,obek_1
...,...
2335,obek_8
2336,obek_2
2337,obek_7


In [19]:
sub_df.to_csv('submission_lgbm_fe2.csv')